In [1]:
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter

In [2]:
stopwords=["अंदर","अत","अदि","अप","अपना","अपनि","अपनी","अपने","अभि","अभी","आदि","आप","इंहिं","इंहें","इंहों","इतयादि","इत्यादि","इन","इनका","इन्हीं","इन्हें","इन्हों","इस","इसका","इसकि","इसकी","इसके","इसमें","इसि","इसी","इसे","उंहिं","उंहें","उंहों","उन","उनका","उनकि","उनकी","उनके","उनको","उन्हीं","उन्हें","उन्हों","उस","उसके","उसि","उसी","उसे","एक","एवं","एस","एसे","ऐसे","ओर","और","कइ","कई","कर","करता","करते","करना","करने","करें","कहते","कहा","का","काफि","काफ़ी","कि","किंहें","किंहों","कितना","किन्हें","किन्हों","किया","किर","किस","किसि","किसी","किसे","की","कुछ","कुल","के","को","कोइ","कोई","कोन","कोनसा","कौन","कौनसा","गया","घर","जब","जहाँ","जहां","जा","जिंहें","जिंहों","जितना","जिधर","जिन","जिन्हें","जिन्हों","जिस","जिसे","जीधर","जेसा","जेसे","जैसा","जैसे","जो","तक","तब","तरह","तिंहें","तिंहों","तिन","तिन्हें","तिन्हों","तिस","तिसे","तो","था","थि","थी","थे","दबारा","दवारा","दिया","दुसरा","दुसरे","दूसरे","दो","द्वारा","न","नहिं","नहीं","ना","निचे","निहायत","नीचे","ने","पर","पहले","पुरा","पूरा","पे","फिर","बनि","बनी","बहि","बही","बहुत","बाद","बाला","बिलकुल","भि","भितर","भी","भीतर","मगर","मानो","मे","में","यदि","यह","यहाँ","यहां","यहि","यही","या","यिह","ये","रखें","रवासा","रहा","रहे","ऱ्वासा","लिए","लिये","लेकिन","व","वगेरह","वरग","वर्ग","वह","वहाँ","वहां","वहिं","वहीं","वाले","वुह","वे","वग़ैरह","संग","सकता","सकते","सबसे","सभि","सभी","साथ","साबुत","साभ","सारा","से","सो","हि","ही","हुअ","हुआ","हुइ","हुई","हुए","हे","हें","है","हैं","हो","होता","होति","होती","होते","होना","होने"]

In [3]:
alphabets=['ऄ','अ','आ','इ','ई','उ','ऊ','ऋ','ऌ','ऍ','ऎ','ए','ऐ','ऑ','ऒ','ओ','औ','क','ख','ग','घ','ङ','च','छ','ज','झ','ञ','ट',
'ठ','ड','ढ','ण','त','थ','द','ध','न','ऩ','प','फ','ब','भ','म','य','र','ऱ','ल','ळ','ऴ','व','श','ष','स','ह']

matra=['ऀ','ँ','ं','ः','ऺ','ऻ','़','ा','ि','ी','ु','ू','ृ','ॄ','ॅ','ॆ','े','ै','ॉ','ॊ','ो','ौ','्','ॎ','ॏ','ॕ','ॖ','ॗ']

In [4]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq
def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)
        
def remove_special_characters(text):
    string=''
    for i in text:
        if i in alphabets+matra:
            string=string+i
        else:
            string=string+" "
    return " ".join(string.split())

In [5]:
data=pd.read_csv('../hindi.csv')

In [6]:
doc_words=[]
dict_global = {}
idx = 0
files_with_index = {}
for index in range(len(data)):
    fname=str(data.loc[index]['Chapter'])+"."+str(data.loc[index]['Verse'])
    sloka=data.loc[index]['Sloke']
    text=data.loc[index]['Commentary']
    text = remove_special_characters(text)
    words = text.split()
    words = [word for word in words if word not in stopwords]
    doc_words.append(words)
    dict_global.update(finding_all_unique_words_and_freq(words))
    files_with_index[idx] = fname
    idx = idx + 1
    
unique_words_all = set(dict_global.keys())
    


In [7]:
tf={}
df={}
for i in unique_words_all:
    tf[i]={}
    df[i]=0   

In [8]:
idx=0
Ltot=0
Ld={}
for index in range(len(data)):
    text=data.loc[index]['Commentary']
    text = remove_special_characters(text)
    words = text.split()
    words = [word for word in words if word not in stopwords]
    Ld[idx]=len(words) #len of current doc
    Ltot=Ltot+len(words) #sum of lens of all the docs
    counter=Counter(words)
    for i in counter.keys():
        df[i]=df[i]+1
        tf[i][idx]=counter[i]   
    idx=idx+1

In [9]:
doc_norm={}
idx=0
for i in doc_words:
    l2=0
    for j in set(i):
        l2+=(i.count(j)*math.log(len(files_with_index)/df[j]))**2
    doc_norm[idx]=(math.sqrt(l2))
    idx+=1

In [10]:
with open('pickles/posting_list.pkl','wb') as file:
    pickle.dump(tf,file)
    file.close()
    
with open('pickles/df.pkl','wb') as file:
    pickle.dump(df,file)
    file.close()
    
with open('pickles/doc_len.pkl','wb') as file:
    pickle.dump(Ld,file)
    file.close()

In [11]:
a_file = open("pickles/file_idx.pkl", "wb")
pickle.dump(files_with_index, a_file)
a_file.close()
a_file = open("pickles/unique_words_all.pkl", "wb")
pickle.dump(unique_words_all , a_file)
a_file.close()

In [12]:
with open('pickles/doc_words.pkl','wb') as file:
    pickle.dump(doc_words,file)
    file.close()
    
with open('pickles/doc_norm.pkl','wb') as file:
    pickle.dump(doc_norm,file)
    file.close()